# Graph Neural Networks Tutorial(GNN)

#### 1. Pytorch Geometric Framework
- Understanding Message Passing Scheme in Pytorch Geometric.
- Efficient graph data representations and paralleling minibatching graphs.
- Showcase the implementation of **Graph Convolution Networks** (Kipf & Welling, [SEMI-SUPERVISED CLASSIFICATION WITH GRAPH CONVOLUTIONAL NETWORKS](https://arxiv.org/abs/1609.02907), ICLR 2017), and you should implement **GraphSAGE** (Hamilton et al, [Inductive Representation Learning on Large Graphs](https://arxiv.org/abs/1706.02216), NIPS 2017) in the lab based on message passing scheme.

#### 2. Vertex Classification
- Showcase a model developed based on our GCN implementation to do vertex classification on Cora dataset. 
- Develop a model with **your own** GraphSAGE (with mean/sum/max aggregation) implementation on the same dataset to get insights of difference.

#### 3. Graph Classification
- Implement **GINConv** (Xu et al, [HOW POWERFUL ARE GRAPH NEURAL NETWORKS?](https://arxiv.org/abs/1810.00826), ICLR 2019) on graph classification benchmark dataset (IMDB) and compare different aggregation functions (SUM/MEAN/MAX).

## Setting up working environment

For this tutorial you will need to train a large network, therefore we recommend you work with Google Colaboratory, which provides free GPU time. You will need a Google account to do so. Please log in to your account and go to the following page: https://colab.research.google.com. Then upload this notebook.For GPU support, go to "Edit" -> "Notebook Settings", and select "Hardware accelerator" as "GPU".You will need to install pytorch by running the following cell:

In [ ]:
!pip install torch torchvision
!pip install torch-scatter torch-sparse torch-geometric

## Pytorch Geometric Framework

#### Generic Message Passing Scheme
Generalizing the convolution operator to irregular domains is typically expressed as a *neighborhood aggregation* or *message passing* scheme.
With $\mathbf{x}^{(k-1)}_i \in \mathbb{R}^F$ denoting node features of node $i$ in layer $(k-1)$ and $\mathbf{e}_{i,j} \in \mathbb{R}^D$ denoting (optional) edge features from node $i$ to node $j$, message passing graph neural networks can be described as

$$
  \mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{i,j}\right) \right)
$$

where $\square$ denotes a differentiable, permutation invariant function, *e.g.*, sum, mean or max, and $\gamma$ and $\phi$ denote differentiable functions such as MLPs (Multi Layer Perceptrons).

#### Graph data representations in PyG
Given a *sparse* **Graph** $\mathcal{G}=(\mathbf{X}, (\mathbf{I}, \mathbf{E}))$ with **node features** $\mathbf{X} \in \mathbb{R}^{|V| \times F}$, **edge indices $\mathbf{I} \in \{1, \cdots, N\}^{2 \times |\mathcal{E}|}$**, (optional) **edge features** $\mathbf{E} \in \mathbb{R}^{|\mathcal{E} \times D|}$, it is described by an instance of class `torch_geometric.data.Data`, which holds the corresponding attributes.

We show a simple example of an unweighted and directed graph with four nodes and three edges.

<p align="center"><img width="70%" src="./figures/graph_data.png"></p>

In [1]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[2, 1, 3],
                           [0, 0, 2]], dtype=torch.long)
x = torch.tensor([[1], [1], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data

Data(x=[3, 1], edge_index=[2, 3])

#### Mini-Batching Graphs
Neural networks are usually trained in a batch-wise fashion. Minibatch graphs can be efficiently dealt with to achieve parallelization over a mini-batch from creating sparse block diagnoal adjacency matrices and concatenating features and target matrices in the node dimension.


<p align="center"><img width="70%" src="./figures/mini_batch_graph.png"></p>

#### Abstract Message Passing Scheme in PyG

PyTorch Geometric provides the `torch_geometric.nn.MessagePassing` base class, which helps in creating such kinds of message passing graph neural networks by automatically taking care of message propagation. The implementation is decoupled into **UPDATE**, **AGGREGATION**, **MESSAGE** functions as:
$$
    \mathbf{x}_i^{(k)} = \mathrm{UPDATE} \left( \mathbf{x}_i, , \mathrm{AGGR}_{j \in \mathcal{N}(i)} \, \mathrm{MESSAGE}^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{i,j}\right) \right)    
$$

<p align="center"><img width="70%" src="./figures/message_passing.png"></p>

#### Implementing the GCN layer (lecture)

The graph convolutional operator introduced by Kipf & Welling (ICLR 2017) is defined as
$$
        \mathbf{X}^{k} = \mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
        \mathbf{\hat{D}}^{-1/2} \mathbf{X}^{k-1} \mathbf{\Theta},
$$
where $\mathbf{\hat{A}} = \mathbf{A} + \mathbf{I}$ denotes the adjacency matrix with inserted self-loops and
$\hat{D}_{ii} = \sum_{j=0} \hat{A}_{ij}$ its diagonal degree matrix. It is equivalent as:
$$
\mathbf{x}_i^{(k)} = \sum_{j \in \mathcal{N}(i) \cup \{ i \}} \frac{1}{\sqrt{\deg(i)} \cdot \sqrt{deg(j)}} \cdot \left( \mathbf{x}_j^{(k-1)}\mathbf{\Theta} \right),
$$

where neighboring node features are first transformed by a weight matrix $\mathbf{\Theta}$, normalized by their degree, and finally summed up.
This formula can be divided into the following steps:

1. Add self-loops to the adjacency matrix.
2. Linearly transform node feature matrix.
3. Normalize node features.
4. Sum up neighboring node features.
5. Return new node embeddings.

In [2]:
import torch
from torch_geometric.nn import MessagePassing
import math

def glorot(tensor):
    if tensor is not None:
        stdv = math.sqrt(6.0 / (tensor.size(-2) + tensor.size(-1)))
        tensor.data.uniform_(-stdv, stdv)


def zeros(tensor):
    if tensor is not None:
        tensor.data.fill_(0)

        
def add_self_loops(edge_index, num_nodes=None):
    loop_index = torch.arange(0, num_nodes, dtype=torch.long,
                              device=edge_index.device)
    loop_index = loop_index.unsqueeze(0).repeat(2, 1)

    edge_index = torch.cat([edge_index, loop_index], dim=1)

    return edge_index


def degree(index, num_nodes=None, dtype=None):
    out = torch.zeros((num_nodes), dtype=dtype, device=index.device)
    return out.scatter_add_(0, index, out.new_ones((index.size(0))))
        

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        
        self.reset_parameters()
        
    def reset_parameters(self):
        glorot(self.lin.weight)
        zeros(self.lin.bias)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        ########################################################################
        #      START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)             #
        ########################################################################
        # Step 1: Add self-loops to the adjacency matrix.
        
        edge_index = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages.

        return self.propagate(edge_index, x=x)
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################                




    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]

        ########################################################################
        #      START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)             #
        ########################################################################

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j        
        
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################              
        


    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

#### Implementing GraphSAGE (lab)

The algorithm of GraphSAGE (*Inductive Representation Learning on Large Graphs (NIPS 2017)*) embedding generation is described as:

<p align="center"><img width="70%" src="./figures/graphsage.png"></p>

You are required to implement this algortihm with **MEAN/SUM/MAX** AGGREGATE.

In [4]:
import torch
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
import math

def uniform(size, tensor):
    """Initializes a tensor with uniform distribution."""
    bound = 1.0 / math.sqrt(size)
    if tensor is not None:
        tensor.data.uniform_(-bound, bound)

class SAGEConv(MessagePassing):
    """
    GraphSAGE layer implementation.

    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        aggr (str, optional): Aggregation method ('add', 'mean', 'max'). Default is 'mean'.
    """
    def __init__(self, in_channels, out_channels, aggr='mean'):
        super(SAGEConv, self).__init__(aggr=aggr)

        self.in_channels = in_channels
        self.out_channels = out_channels

        # Weight parameter for feature transformation
        self.weight = Parameter(torch.Tensor(2 * in_channels, out_channels))

        # Reset parameters
        self.reset_parameters()

    def reset_parameters(self):
        """Resets parameters using uniform initialization."""
        uniform(self.weight.size(0), self.weight)

    def forward(self, x, edge_index):
        """
        Forward pass of the GraphSAGE layer.

        Args:
            x (torch.Tensor): Node feature matrix of shape [num_nodes, in_channels].
            edge_index (torch.Tensor): Edge indices in COO format with shape [2, num_edges].
        
        Returns:
            torch.Tensor: Updated node features of shape [num_nodes, out_channels].
        """
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        """
        Constructs messages from neighboring nodes.

        Args:
            x_j (torch.Tensor): Neighbor features of shape [num_edges, in_channels].
        
        Returns:
            torch.Tensor: Message tensor of shape [num_edges, in_channels].
        """
        return x_j

    def update(self, aggr_out, x):
        """
        Updates node features after message aggregation.

        Args:
            aggr_out (torch.Tensor): Aggregated messages of shape [num_nodes, in_channels].
            x (torch.Tensor): Original node features of shape [num_nodes, in_channels].
        
        Returns:
            torch.Tensor: Updated node features of shape [num_nodes, out_channels].
        """
        # Concatenate original node features with aggregated features
        aggr_out = torch.cat([x, aggr_out], dim=-1)

        # Apply linear transformation
        aggr_out = torch.matmul(aggr_out, self.weight)

        # Normalize features
        aggr_out = F.normalize(aggr_out, p=2, dim=-1)

        return aggr_out


## Vertex Classification

In [5]:
import os
import os.path as osp
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

path = osp.join(os.getcwd(), 'data', 'Cora')
dataset = Planetoid(path, 'Cora')

Processing...
Done!


In [9]:
import os
import torch
from torch_geometric.datasets import Planetoid

# Define the dataset path dynamically
def get_dataset_path(dataset_name='Cora', base_dir='data'):
    """Returns the dataset storage path."""
    return os.path.join(os.getcwd(), base_dir, dataset_name)

# Load the dataset
def load_dataset(dataset_name='Cora'):
    """
    Loads the specified Planetoid dataset.
    
    Args:
        dataset_name (str): Name of the dataset to load (e.g., 'Cora', 'Citeseer').
        
    Returns:
        torch_geometric.data.Dataset: The loaded dataset.
    """
    path = get_dataset_path(dataset_name)
    return Planetoid(path, dataset_name)

# Load Cora dataset
dataset = load_dataset('Cora')
print(f"Dataset: {dataset}")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of classes: {dataset.num_classes}")
print(f"Number of node features: {dataset.num_node_features}")


Dataset: Cora()
Number of graphs: 1
Number of classes: 7
Number of node features: 1433


In [10]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.optim import Adam
import time

# Hyperparameters
runs = 10
epochs = 200
lr = 0.01
weight_decay = 0.0005
early_stopping = 10
hidden = 16
dropout = 0.5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Net(torch.nn.Module):
    def __init__(self, dataset):
        """
        Define the layers and initialization of the GCN model.
        
        Args:
            dataset (torch_geometric.data.Dataset): The input graph dataset.
        """
        super(Net, self).__init__()
        
        # Define the GCN layers
        self.conv1 = GCNConv(dataset.num_features, hidden)
        self.conv2 = GCNConv(hidden, dataset.num_classes)

    def reset_parameters(self):
        """Reset parameters of the layers."""
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, data):
        """
        Forward pass through the model.
        
        Args:
            data (torch_geometric.data.Data): The input graph data containing node features and edge indices.
        
        Returns:
            torch.Tensor: The log-softmax probabilities for each node.
        """
        x, edge_index = data.x, data.edge_index
        
        # Apply the first GCN layer followed by ReLU activation
        x = F.relu(self.conv1(x, edge_index))
        
        # Apply dropout for regularization
        x = F.dropout(x, p=dropout, training=self.training)
        
        # Apply the second GCN layer
        x = self.conv2(x, edge_index)
        
        # Return log-softmax values for classification
        return F.log_softmax(x, dim=1)

# Running the model with hyperparameters and dataset
run(dataset, Net(dataset).to(device), runs, epochs, lr, weight_decay, early_stopping)


Val Loss: 0.7261, Test Accuracy: 0.785 ± 0.011, Duration: 5.545
